In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MinhaAplicacao").getOrCreate()
sc = spark.sparkContext

25/05/15 15:54:25 WARN Utils: Your hostname, sg resolves to a loopback address: 127.0.1.1; using 10.100.25.210 instead (on interface wlo1)
25/05/15 15:54:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/15 15:54:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc

<SparkContext master=local[*] appName=MinhaAplicacao>

In [3]:
rdd = sc.textFile("pacientes.csv")

In [7]:
rdd.take(1)

['PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show']

In [5]:
%%time
rdd.count()

CPU times: user 11.2 ms, sys: 5.91 ms, total: 17.2 ms
Wall time: 1.48 s


110528

In [8]:
df = spark.read.csv("pacientes.csv", header=True, inferSchema=True)
df.show(5)

+-------------------+-------------+------+-------------------+-------------------+---+-----------------+-----------+------------+--------+----------+-------+------------+-------+
|          PatientId|AppointmentID|Gender|       ScheduledDay|     AppointmentDay|Age|    Neighbourhood|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|SMS_received|No-show|
+-------------------+-------------+------+-------------------+-------------------+---+-----------------+-----------+------------+--------+----------+-------+------------+-------+
| 2.9872499824296E13|      5642903|     F|2016-04-29 15:38:08|2016-04-28 21:00:00| 62|  JARDIM DA PENHA|          0|           1|       0|         0|      0|           0|     No|
|5.58997776694438E14|      5642503|     M|2016-04-29 13:08:27|2016-04-28 21:00:00| 56|  JARDIM DA PENHA|          0|           0|       0|         0|      0|           0|     No|
|  4.262962299951E12|      5642549|     F|2016-04-29 13:19:04|2016-04-28 21:00:00| 62|    MATA DA PRAIA| 

In [10]:
from pyspark.sql.functions import col

df = df.withColumn("PatientId", col("PatientId").cast("long"))

In [11]:
df.show(50)

+---------------+-------------+------+-------------------+-------------------+---+-----------------+-----------+------------+--------+----------+-------+------------+-------+
|      PatientId|AppointmentID|Gender|       ScheduledDay|     AppointmentDay|Age|    Neighbourhood|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|SMS_received|No-show|
+---------------+-------------+------+-------------------+-------------------+---+-----------------+-----------+------------+--------+----------+-------+------------+-------+
| 29872499824296|      5642903|     F|2016-04-29 15:38:08|2016-04-28 21:00:00| 62|  JARDIM DA PENHA|          0|           1|       0|         0|      0|           0|     No|
|558997776694438|      5642503|     M|2016-04-29 13:08:27|2016-04-28 21:00:00| 56|  JARDIM DA PENHA|          0|           0|       0|         0|      0|           0|     No|
|  4262962299951|      5642549|     F|2016-04-29 13:19:04|2016-04-28 21:00:00| 62|    MATA DA PRAIA|          0|           0|

In [12]:
df.printSchema()

root
 |-- PatientId: long (nullable = true)
 |-- AppointmentID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- ScheduledDay: timestamp (nullable = true)
 |-- AppointmentDay: timestamp (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Neighbourhood: string (nullable = true)
 |-- Scholarship: integer (nullable = true)
 |-- Hipertension: integer (nullable = true)
 |-- Diabetes: integer (nullable = true)
 |-- Alcoholism: integer (nullable = true)
 |-- Handcap: integer (nullable = true)
 |-- SMS_received: integer (nullable = true)
 |-- No-show: string (nullable = true)



In [13]:
spark.sql("DROP DATABASE IF EXISTS hospital")

DataFrame[]

In [14]:
spark.sql("CREATE DATABASE IF NOT EXISTS hospital")

DataFrame[]

In [15]:
df.write.mode("overwrite").saveAsTable("hospital.pacientes")

In [17]:
%%time
df_pac = spark.read.table("hospital.pacientes")
df_pac.count()

CPU times: user 950 μs, sys: 991 μs, total: 1.94 ms
Wall time: 671 ms


110527

In [ ]:
df_pac.show(3)

In [ ]:
df_pac.show(5)

In [20]:
query = """
-- Contar pacientes por Gender
SELECT Gender,
        count(*) AS count
FROM hospital.pacientes
GROUP BY Gender
"""
df2 = spark.sql(query)
df2.show(5)

+------+-----+
|Gender|count|
+------+-----+
|     F|71840|
|     M|38687|
+------+-----+

